In [19]:
import pandas as pd
import numpy as np
import os

symptom_precaution = pd.read_csv(os.path.join("datasets", "disease symptom prediction", "symptom_precaution.csv"))
symptom_desc = pd.read_csv(os.path.join("datasets", "disease symptom prediction", "symptom_Description.csv"))
symptoms = pd.read_csv(os.path.join("datasets", "disease symptom prediction", "Symptom-severity.csv")).to_numpy()

In [20]:
data = pd.read_csv(os.path.join("datasets", "disease symptom prediction", "dataset.csv"))
arr = data.to_numpy()

allInflictions = set()
for row in arr:
    row = row[1:]
    for item in row:
        if type(item) is str and item not in allInflictions:
            allInflictions.add(item.strip())

allInflictions

{'abdominal_pain',
 'abnormal_menstruation',
 'acidity',
 'acute_liver_failure',
 'altered_sensorium',
 'anxiety',
 'back_pain',
 'belly_pain',
 'blackheads',
 'bladder_discomfort',
 'blister',
 'blood_in_sputum',
 'bloody_stool',
 'blurred_and_distorted_vision',
 'breathlessness',
 'brittle_nails',
 'bruising',
 'burning_micturition',
 'chest_pain',
 'chills',
 'cold_hands_and_feets',
 'coma',
 'congestion',
 'constipation',
 'continuous_feel_of_urine',
 'continuous_sneezing',
 'cough',
 'cramps',
 'dark_urine',
 'dehydration',
 'depression',
 'diarrhoea',
 'dischromic _patches',
 'distention_of_abdomen',
 'dizziness',
 'drying_and_tingling_lips',
 'enlarged_thyroid',
 'excessive_hunger',
 'extra_marital_contacts',
 'family_history',
 'fast_heart_rate',
 'fatigue',
 'fluid_overload',
 'foul_smell_of urine',
 'headache',
 'high_fever',
 'hip_joint_pain',
 'history_of_alcohol_consumption',
 'increased_appetite',
 'indigestion',
 'inflammatory_nails',
 'internal_itching',
 'irregular_sug

In [21]:
counter = 0
indexer = {}
for item in allInflictions:
    indexer[item] = counter
    counter += 1

indexer

{'yellowish_skin': 0,
 'puffy_face_and_eyes': 1,
 'history_of_alcohol_consumption': 2,
 'dark_urine': 3,
 'ulcers_on_tongue': 4,
 'internal_itching': 5,
 'pain_during_bowel_movements': 6,
 'scurring': 7,
 'distention_of_abdomen': 8,
 'phlegm': 9,
 'blood_in_sputum': 10,
 'enlarged_thyroid': 11,
 'muscle_weakness': 12,
 'painful_walking': 13,
 'weakness_in_limbs': 14,
 'irritation_in_anus': 15,
 'red_sore_around_nose': 16,
 'stiff_neck': 17,
 'continuous_sneezing': 18,
 'burning_micturition': 19,
 'swelling_of_stomach': 20,
 'mood_swings': 21,
 'loss_of_smell': 22,
 'chills': 23,
 'skin_peeling': 24,
 'swollen_legs': 25,
 'watering_from_eyes': 26,
 'yellowing_of_eyes': 27,
 'weight_gain': 28,
 'silver_like_dusting': 29,
 'throat_irritation': 30,
 'rusty_sputum': 31,
 'itching': 32,
 'drying_and_tingling_lips': 33,
 'restlessness': 34,
 'sinus_pressure': 35,
 'muscle_wasting': 36,
 'visual_disturbances': 37,
 'irregular_sugar_level': 38,
 'obesity': 39,
 'loss_of_appetite': 40,
 'swelled

In [22]:
structured = [] # 2D array of data [[0s/1s], [diseasename]]
for row in arr:
    infliction = row[0].strip()
    row = row[1:]
    newRow = [0 for i in range(counter)]
    for symptom in row:
        if type(symptom) is str:
            newRow[indexer[symptom.strip()]] = 1
    structured.append([newRow, infliction])

df = pd.DataFrame(structured)
df

,0,1
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Fungal infection
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Fungal infection
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Fungal infection
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Fungal infection
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Fungal infection
...,...,...
4915,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",(vertigo) Paroymsal Positional Vertigo
4916,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",Acne
4917,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Urinary tract infection
4918,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Psoriasis


In [23]:
ordered_list = sorted((value, key) for key, value in indexer.items())
ordered_words_only = [item[1].strip() for item in ordered_list]

In [24]:
learning_data = df.to_numpy()

unique_int_to_illness = {k: v for k, v in enumerate(symptom_desc.to_numpy()[:, 0])}
illness_to_unique_int = {v: k for k, v in unique_int_to_illness.items()}
map_values = np.vectorize(lambda x : illness_to_unique_int[x])

learning_data[:, 1] = map_values(learning_data[:, 1])
learning_data

array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        27],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        27],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [25]:
X = list(learning_data[:, 0])
Y = list(learning_data[:, 1])

In [26]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression().fit(X, Y)

In [27]:
unique_int_to_illness[classifier.predict([X[0]])[0]]

'Paralysis (brain hemorrhage)'